In [33]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from numpy.linalg import norm

import sys
sys.path.append('..')

import modules.mean_shift as ms
import modules.general as gen
import modules.plotting as pl
import modules.pandas_funcs as pf
import modules.pose_estimation as pe
import modules.signals as sig
import modules.gait_metrics as gm
import modules.linear_algebra as lin
import modules.point_processing as pp

In [2]:
%matplotlib auto

Using matplotlib backend: MacOSX


In [3]:
directory = os.path.join('..', 'data', 'pickles')

file_name = '2014-12-16_P005_Pre_002_df_pass.pkl'
file_path = os.path.join(directory, file_name)

df_pass = pd.read_pickle(file_path)
frames = df_pass.index.values

## Cluster foot distance 

In [4]:
foot_dist = (df_pass.L_FOOT - df_pass.R_FOOT).apply(norm)

In [5]:
masses = foot_dist.values

labels, centroids, k = ms.cluster(frames.reshape(-1, 1), masses=masses, kernel='gaussian', radius=5)

In [37]:
points = foot_dist.reset_index().values

plt.figure()
pl.scatter_labels(points, labels)
plt.title('Foot distance')
plt.grid()
plt.show()

In [23]:
def make_consistent(df_cluster):
    
    df_assigned = df_cluster.copy()
            
    head_points = np.stack(df_cluster.HEAD)
    
    # Establish a motion correspondence so the foot sides do not switch abruptly
    foot_points_l, foot_points_r = pp.track_two_objects(df_cluster.L_FOOT, df_cluster.R_FOOT)
    
    df_assigned.L_FOOT = pf.series_of_rows(foot_points_l, index=df_cluster.index)
    df_assigned.R_FOOT = pf.series_of_rows(foot_points_r, index=df_cluster.index)
    
    side_values = pe.evaluate_foot_side(head_points, foot_points_l, foot_points_r)
    
    if sum(side_values) > 0:
        
        # The left foot should be labelled the right foot, and vice versa
        df_assigned = pf.swap_columns(df_assigned, 'L_FOOT', 'R_FOOT')
    
    return df_assigned

In [8]:
def enforce_consistency(df_pass):
    
    foot_dist = (df_pass.L_FOOT - df_pass.R_FOOT).apply(norm)

    masses = foot_dist.values
    labels, centroids, k = ms.cluster(frames.reshape(-1, 1), masses=masses, kernel='gaussian', radius=5)
    
    labels = gen.map_sort(labels)
    clustered_dfs = list(gen.group_by_label(df_pass, labels))
    
    consistent_dfs = [assign_sides(x) for x in clustered_dfs]
    
    return pd.concat(consistent_dfs)

In [14]:
foot_dist = (df_pass.L_FOOT - df_pass.R_FOOT).apply(norm)

masses = foot_dist.values
labels, centroids, k = ms.cluster(frames.reshape(-1, 1), masses=masses, kernel='gaussian', radius=5)

labels = gen.map_sort(labels)
clustered_dfs = list(gen.group_by_label(df_pass, labels))

In [24]:
df_cluster = make_consistent(clustered_dfs[2])

In [10]:
_, direction_pass = pe.direction_of_pass(df_pass_cons)

signal_left = gm.foot_signal(df_pass_cons.L_FOOT, df_pass_cons.R_FOOT, direction_pass)

plt.figure()
plt.plot(signal_left)
plt.show()

## View multiple frames

In [25]:
df = df_cluster

In [36]:
fig, ax = plt.subplots()

ax = Axes3D(fig)

for frame in range(300, 370):
    
    plt.cla()
    
    try:
        positions = df.loc[frame]
    except KeyError:
        continue
    
    points = np.stack(positions)
    points_to_plot = points[:, [0, 2, 1]]

    head, foot_l, foot_r = points_to_plot

    pl.scatter3(ax, head, color='k')
    pl.scatter3(ax, foot_l, color='b')
    pl.scatter3(ax, foot_r, color='r')    
    
    ax.set_xlim3d(-200, 200)
    ax.set_ylim3d(100, 300)
    ax.set_zlim3d(-100, 100)
    
    plt.title('Frame {}'.format(frame))
    
    plt.pause(0.1)

### View one frame

In [17]:
frame = 340

fig, ax = plt.subplots()

ax = Axes3D(fig)

positions = df_assigned.loc[frame]

points = np.stack(positions)
points_to_plot = points[:, [0, 2, 1]]

head, foot_l, foot_r = points_to_plot

pl.scatter3(ax, head, color='k')
pl.scatter3(ax, foot_l, color='b')
pl.scatter3(ax, foot_r, color='r')    

ax.set_xlim3d(-200, 200)
ax.set_ylim3d(100, 300)
ax.set_zlim3d(-100, 100)

plt.show()

In [22]:
foot_dist = (df_pass.L_FOOT - df_pass.R_FOOT).apply(np.linalg.norm)

In [23]:
plt.plot(foot_dist)